# The Price is Right

Today we'll build another piece of the puzzle: a ScanningAgent that looks for promising deals by subscribing to RSS feeds.

In [16]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection

In [17]:
# Initialize and constants

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [18]:
deals = ScrapedDeal.fetch(show_progress=True)

100%|██████████| 5/5 [05:44<00:00, 68.87s/it]


In [19]:
len(deals)

50

In [20]:
deals[44].describe()

"Title: Vevor Tufting Gun for $73 + free shipping\nDetails: It's the best price we could find by $40. Create a free account to get this price and to have this item shipped for free, although if you live in a particular rural area, you may have to pay for shipping. Buy Now at VEVOR\nFeatures: 70W motor can reach 7-27 stiches per second rotating adjustable handle Model: GR-1\nURL: https://www.dealnews.com/products/Vevor/Vevor-Tufting-Gun/487014.html?iref=rss-c196"

In [21]:
system_prompt = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}"""

In [22]:
user_prompt = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [23]:
print(user_prompt[:2000])

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: Chargeworx Dual USB Slim 10,000mAh Power Bank for $10 + free shipping
Details: It's $30 off and the best price we could find. Buy Now at MorningSave
Features: multiple USB ports for charging up to two devices at a time
URL: https://www.dealnews.com/Chargeworx-Dual-USB-Slim-10-000-m-Ah-Power-Bank-for-10-free-shipping/21717611.html?iref=rss-c142

Title: Samsung Presid

In [24]:
def get_recommendations():
    completion = openai.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
      ],
        response_format=DealSelection
    )
    result = completion.choices[0].message.parsed
    return result

In [25]:
result = get_recommendations()

In [26]:
len(result.deals)

5

In [ ]:
result.deals[1]

Deal(product_description='The Samsung QE1D QN75QE1DAFXZA 75" QLED 4K Smart TV provides stunning image quality with its 4K resolution and advanced QLED technology, allowing for vibrant colors and deep contrasts. It features smart capabilities for accessing your favorite apps and services seamlessly. This premium display is complemented by a sleek design that enhances the aesthetics of any room.', price=980.0, url='https://www.dealnews.com/Samsung-Presidents-Day-TV-and-TV-Bundle-Deals-Up-to-2-018-off-up-to-540-trade-in-credit-free-shipping/21717542.html?iref=rss-c142')

In [28]:
from agents.scanner_agent import ScannerAgent

In [29]:
agent = ScannerAgent()
result = agent.scan()

In [30]:
result

DealSelection(deals=[Deal(product_description="The Snorecircle Snore Earset E4 is an innovative anti-snoring device designed to help users achieve a more restful night's sleep. This earset utilizes vibration intervention technology that detects snoring and gently vibrates to prompt the user to adjust their sleeping position. It offers adjustable vibration intensity to suit individual preferences and includes features to provide daily sleep reports, allowing users to track their progress over time. With up to two nights of use on a single charge, the Snorecircle Earset is both convenient and effective for snoring relief.", price=69.0, url='https://www.dealnews.com/products/Snorecircle-Snore-Earset-E4-Anti-Snoring-Device/241886.html?iref=rss-c142'), Deal(product_description='The iBuyPower Trace Mesh Desktop is a high-performance gaming rig powered by an AMD Ryzen 7 processor, specifically the 5700, which runs at 3.7GHz with 8 cores. It features the NVIDIA GeForce RTX 4060 GPU, ensuring t